In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import nibabel as nib
import numpy as np 
# import pandas as pd
# import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from nibabel.testing import data_path
# from skimage import feature
from scipy import ndimage
import glob
import cv2

CUDA_VISIBLE_DEVICES=0

def normimg(input_image):
    input_image = np.around(input_image)
    input_image = input_image / 2286.0
    return input_image

# rotation de l'image a fin d'avoir plus d'images ( data ogmentation )
def rot_aug(input_image):
    input_image = ndimage.rotate(input_image, -5, reshape=False)
    return input_image

def flip_aug(input_image):
    input_image = np.flipud(input_image)
    return input_image

def shift_aug(input_image):
    input_image = ndimage.shift(input_image[:, :, 0], (3, -20))
    input_image = np.expand_dims(input_image, axis=-1)
    return input_image

def load_data(data_path):
  data = sorted(glob.glob(data_path))
  total=len(data)
  imag = []
  masks = []
  for count, file in enumerate(data,1):

    image = nib.load(file).get_fdata()[:,:,:,0]
    imag.append(image)  

    print("{} / {}".format(count,total))
  return np.array(imag)



# path to image input
DATA_PATH ='../Inputs/Nifti_images/CT_*.nii.gz'

IMAGES = load_data(DATA_PATH)
print('dimensions: ',IMAGES.shape,' ; ','Type des images: ', type(IMAGES))
# print('dimensions: ',IMAGES.shape,' ; ','Type des images: ', type(IMAGES),' ; ', 'coder sur: ', IMAGES[1].dtype)

# path to model
# new_model = tf.keras.models.load_model('/src/Skin_Segmentation/Weights/model_for_medic.h5')
new_model = tf.keras.models.load_model('../Weights/model_for_medic.h5')

# Save Predictions
def predict_data(data_path):
  data = sorted(glob.glob(data_path))
  total=len(data)

  for count, file in enumerate(data,1):
    imags = []
    image = nib.load(file)
    im = normimg(image.get_fdata()[:,:,:,0])
    imags.append(im)
    test=  np.array(imags)
    resultat = new_model.predict(test)
    for i in resultat :
      i = i[:,:] * 5
      i = np.around(i)
      a=i
      i = i[:,:] * 51
      nft_img = nib.Nifti1Image(a, image.affine)
      nib.save(nft_img, os.path.join('../Outputs/Nifti_Outputs/Output_img%01.0d.nii.gz'%count ))
      cv2.imwrite('../Outputs/JPEG_Outputs/Output_img%01.0d.jpeg'%count, i)
      print("{} / {}".format(count,total))
      return np.array(imags)

DATA_PATH ='../Inputs/Nifti_images/CT_*.nii.gz'

predict_data(DATA_PATH)


# # Nouvelle section

2021-07-08 18:33:57.144512: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-07-08 18:33:57.144548: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


1 / 3
2 / 3
3 / 3
dimensions:  (3, 384, 384, 1)  ;  Type des images:  <class 'numpy.ndarray'>


2021-07-08 18:33:58.657627: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-08 18:33:58.657680: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-08 18:33:58.657712: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5b754bfe4e4d): /proc/driver/nvidia/version does not exist
2021-07-08 18:33:58.658145: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-08 18:33:58.685000: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 240000000

1 / 3


array([[[[0.32677165],
         [0.33245844],
         [0.3368329 ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.32764654],
         [0.33070866],
         [0.33289589],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.32939633],
         [0.32895888],
         [0.33027122],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.31102362],
         [0.30446194],
         [0.30358705],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.30577428],
         [0.29790026],
         [0.29483815],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.30183727],
         [0.29352581],
         [0.28915136],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]]])

# Nouvelle section